In [36]:
import os
import numpy as np
from wave import open as open_wave # for reading wave files
import librosa
import parselmouth
import seaborn as sns
from scipy import stats
import shutil

In [37]:
raw_data = "raw_data/"
processed_data = "processed_data/"
def fetch_voice_data(url=raw_data): 
    audios = []
    for audio in os.listdir(raw_data):
        if audio.endswith('.wav'):
            audios.append(audio)
    return audios

audios = fetch_voice_data()
print(str(len(audios)) + " audio files loaded")

def npy_to_samples(numpy_array, sample_rate, sample_length, sample_count):
    length = len(numpy_array)/sample_rate
    print(str(length) + " s")
    if len(numpy_array) < sample_rate*sample_length*sample_count:
        return np.array([])
    numpy_array = numpy_array[0:int(sample_rate * sample_length * sample_count)]
    samples = np.empty((0,int(sample_rate * sample_length)), int)
    for i in range(sample_count):
        sample = numpy_array[0:int(sample_rate * sample_length)]
        numpy_array = numpy_array[int(sample_rate * sample_length):]
        samples = np.vstack((samples, sample))
    return samples

def delete_silence(y):
    intervals = librosa.effects.split(y, top_db = 18)
    non_silent_audio = np.zeros(y.shape)
    for i in intervals:
        start, end = i
        non_silent_audio[start:end] = y[start:end]
    non_silent_audio = non_silent_audio[(non_silent_audio != 0)]
    return non_silent_audio

15 audio files loaded


In [38]:
if os.path.exists("processed_data"):
    shutil.rmtree("processed_data")
if not os.path.exists("processed_data"):
    os.makedirs("processed_data")
sns.set() # Use seaborn's default style to make attractive graphs
y = [None] * len(audios)
sr = [None] * len(audios)
rows = []
for i in range(len(audios)):
    # load the file
    print(audios[i])
    y[i], sr[i] = librosa.load(raw_data + audios[i], mono = True, sr = 44100)
    
    # get the label
    label = audios[i].split(".")
    label.pop()
    label = "".join(label)
    
    # remove silent parts
    y[i] = delete_silence(y[i])
    
    # cut into pieces
    sample_count = 50
    sample_length = 0.2 # in seconds
    samples = npy_to_samples(y[i], 44100, sample_length, sample_count)
    if len(samples) > 0:
        if not os.path.exists(processed_data + label):
            os.makedirs(processed_data + label)
        for j in range(sample_count):
            librosa.output.write_wav(processed_data + label + "/" + label + str(j) + ".wav", y[i], 44100)

Bloopers.wav
6.246167800453515 s
Daniel.wav
53.73219954648526 s
Dorothy.wav
8.890249433106575 s
Emily.wav
7.701678004535148 s
Eva.wav
38.620770975056686 s
Jakub.wav
30.735351473922904 s
kahn.wav
10.096893424036281 s
Lei.wav
0.8241723356009071 s
levi.wav
8.17342403628118 s
LudvigPartybox.wav
34.50485260770975 s
Margarita.wav
5.118911564625851 s
Nancy.wav
5.070816326530612 s
ReneElevator.wav
7.186575963718821 s
Richard.wav
73.57535147392291 s
Robert.wav
0.7081179138321996 s
